In [39]:
import pandas as pd
from geopy.distance import geodesic
DISTANCE_THRESHOLD = 3000

In [44]:
# 只保留一种单位的数据
df_mp = pd.read_csv("USA/MicroPlastic.csv")
df_mp = df_mp[df_mp["Unit"] == "pieces/m3"]
df_mp.drop(columns=["Unit"], inplace=True)
df_mp["Date"] = pd.to_datetime(df_mp["Date"])
df_mp['Year'] = df_mp['Date'].dt.year
df_mp['Month'] = df_mp['Date'].dt.month
df_mp.drop(columns=["Date"], inplace=True)
df_mp = df_mp.rename(columns={
	"Latitude":"latitude",
	"Longitude":"longitude",
	"Year":"mp_year",
	"Month":"mp_month"
})
df_mp.describe()

C:\Users\cyo01\AppData\Local\Temp\ipykernel_40300\2584152911.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_mp["Date"] = pd.to_datetime(df_mp["Date"])


,Measurement,latitude,longitude,mp_year,mp_month
count,16031.000000,16031.000000,16031.000000,16031.000000,16031.000000
mean,394.275310,24.621628,-59.791520,2007.238475,7.101054
std,8396.052594,20.663678,75.521488,9.427516,3.424548
min,0.000000,-71.699040,-179.994200,1972.000000,1.000000
25%,0.000000,17.057850,-110.174193,2000.000000,4.000000
50%,0.010000,25.940000,-69.380000,2009.000000,7.000000
75%,0.150748,36.459367,-31.302500,2015.000000,10.000000
max,800000.000000,89.761400,179.854333,2022.000000,12.000000


In [41]:
def is_within_period(row1, row2, period):
	# row1 是出生数据, row2是微塑料数据
	# period的取值为 0 ~ 4. 0对应怀孕前三月，4对应怀孕后三月
	year_diff = row1["birth_year"] - row2["mp_year"]
	month_diff = row1["birth_month"] - row2["mp_month"]
	diff = year_diff * 12 + month_diff
	left = 9 - 3 * period
	right = 11 - 3 * period
	return left <= diff <= right

In [42]:
def is_within_distance(row1, row2, max_distance):
	point1 = (row1["latitude"], row1["longitude"])
	point2 = (row2["latitude"], row2["longitude"])
	return geodesic(point1, point2).km <= max_distance

In [ ]:
df1 = pd.read_csv("USA_LBW.csv")
df2 = df_mp

def myCondition(row, row_in_df1, period):
	return is_within_distance(row_in_df1, row, DISTANCE_THRESHOLD) and is_within_period(row_in_df1, row, period)


# 计算函数
def myJoin(row, df2, period):
	row_in_df1 = row
	filtered_df2 = df2[df2.apply(myCondition, axis=1, args=(row_in_df1, period))]
	result = filtered_df2["Measurement"].avg()
	return result

# 在 df1 中添加新列，通过 apply 来应用计算函数
for i in range(5):
	df1[f'mp{i+1}'] = df1.apply(lambda row: myJoin(row, df2, i), axis=1)

# 打印结果
print(df1)
